In [6]:
.libPaths(c(.libPaths(), '/nfs/apps/lib/R/4.2-focal/site-library.2023q4'))
library(Biostrings)
library(plyr)
library(dplyr)
library(ggplot2)
library(tidyr)
library(ggpubr)
library(metap)
library(ggrepel)
library(stringr)
library(RColorBrewer)
library(VennDiagram)
library(diceR)
library(Rcpp)

In [5]:
# unzip allele table
allele_tables <- list.files(path = "data", pattern = 'Alleles_frequency_table.zip', recursive = TRUE
                            , full.names = TRUE)
for (i in allele_tables) {
    system(paste0("unzip ", i, " -d ", dirname(i)))
}

In [6]:
# merge allele tables
allele_tables <- list.files(pattern = 'Alleles_frequency_table.txt', recursive = TRUE)
allele_table_all <- data.frame(matrix(nrow = 0, ncol = 0))
for (i in allele_tables) {
    tmp <- read.delim(i, stringsAsFactors = FALSE)
    tmp$sample <- gsub(".*/CRISPResso_on_pegRNAlib_", "", dirname(i))
    allele_table_all <- rbind(allele_table_all, tmp)
}

# trim reads - trim more/only consider a certain window around editing site?
allele_table_all$Aligned_Sequence <- substring(allele_table_all$Aligned_Sequence, 5)

# merge and combine reads after trimming
allele_table_summary <- allele_table_all %>% group_by(sample, Aligned_Sequence, n_deleted, n_inserted, n_mutated) %>% 
    summarize(Reads = sum(X.Reads))
allele_table_summary <- allele_table_summary[order(allele_table_summary$Reads, decreasing = TRUE),]

`summarise()` has grouped output by 'sample', 'Aligned_Sequence', 'n_deleted',
'n_inserted'. You can override using the `.groups` argument.


In [8]:
allele_table_summary$Aligned_Sequence_20bpwindow <- gsub("-", "", ifelse(grepl("plasmid_HEK3", allele_table_summary$sample), 
                                            substring(allele_table_summary$Aligned_Sequence, 49)
                                         , ifelse(grepl("plasmid_EMX1", allele_table_summary$sample), 
                                         substring(allele_table_summary$Aligned_Sequence, 27),
                                        ifelse(grepl("plasmid_RNF2", allele_table_summary$sample), 
                                         substring(allele_table_summary$Aligned_Sequence, 29),
                                        ifelse(grepl("gDNA_HEK3", allele_table_summary$sample), 
                                         substring(allele_table_summary$Aligned_Sequence, 168),
                                        ifelse(grepl("gDNA_EMX1", allele_table_summary$sample), 
                                         substring(allele_table_summary$Aligned_Sequence, 159),
                                        ifelse(grepl("gDNA_RNF2", allele_table_summary$sample), 
                                         substring(allele_table_summary$Aligned_Sequence, 30),
                                         NA)))))))
allele_table_summary$Aligned_Sequence_20bpwindow <- substring(allele_table_summary$Aligned_Sequence_20bpwindow, 1, 25)

In [9]:
allele_table_summary_20bp_window <- allele_table_summary %>% 
    group_by(sample, Aligned_Sequence_20bpwindow, n_deleted, n_inserted, n_mutated) %>% 
    summarize(Reads = sum(Reads))
allele_table_summary_20bp_window <- allele_table_summary_20bp_window[
    order(allele_table_summary_20bp_window$Reads, decreasing = TRUE),]

`summarise()` has grouped output by 'sample', 'Aligned_Sequence_20bpwindow',
'n_deleted', 'n_inserted'. You can override using the `.groups` argument.


In [10]:
# annotate either as 5bp insertion or unedited
allele_table_summary_20bp_window$annotation <- ifelse(allele_table_summary_20bp_window$n_inserted == 0 & 
                                          allele_table_summary_20bp_window$n_deleted == 0 & 
                                          allele_table_summary_20bp_window$n_mutated < 2, "unedited",
                                          ifelse((grepl("plasmid_HEK3", allele_table_summary_20bp_window$sample) & 
                                                  grepl("TCTGCCATCA[ACGT]{5}CGTGCTCAGT", 
                                                        allele_table_summary_20bp_window$Aligned_Sequence_20bpwindow)) |
                                                 (grepl("plasmid_EMX1", allele_table_summary_20bp_window$sample) & 
                                                  grepl("GTGATGGGAG[ACGT]{5}TTCTTCTGCT", 
                                                        allele_table_summary_20bp_window$Aligned_Sequence_20bpwindow)) | 
                                                 (grepl("plasmid_RNF2", allele_table_summary_20bp_window$sample) & 
                                                  grepl("AACACCTCAG[ACGT]{5}GTAATGACTA", 
                                                        allele_table_summary_20bp_window$Aligned_Sequence_20bpwindow)) | 
                                                 (grepl("gDNA_EMX1", allele_table_summary_20bp_window$sample) & 
                                                  grepl("AGCAGAAGAA[ACGT]{5}CTCCCATCAC", 
                                                        allele_table_summary_20bp_window$Aligned_Sequence_20bpwindow)) |
                                                 (grepl("gDNA_HEK3", allele_table_summary_20bp_window$sample) & 
                                                  grepl("ACTGAGCACG[ACGT]{5}TGATGGCAGA", 
                                                        allele_table_summary_20bp_window$Aligned_Sequence_20bpwindow)) | 
                                                 (grepl("gDNA_RNF2", allele_table_summary_20bp_window$sample) & 
                                                  grepl("TAGTCATTAC[ACGT]{5}CTGAGGTGTT", 
                                                        allele_table_summary_20bp_window$Aligned_Sequence_20bpwindow))
                                                 , "5bp_insertion","other"))

# extract 5 bp insertion sequence
allele_table_summary_20bp_window$insertion <- ifelse(allele_table_summary_20bp_window$annotation == "5bp_insertion" & 
                                         grepl("plasmid_HEK3", allele_table_summary_20bp_window$sample), 
                                         substring(allele_table_summary_20bp_window$Aligned_Sequence_20bpwindow, 11, 15)
                                         , ifelse(allele_table_summary_20bp_window$annotation == "5bp_insertion" & 
                                         grepl("plasmid_EMX1", allele_table_summary_20bp_window$sample), 
                                         substring(allele_table_summary_20bp_window$Aligned_Sequence_20bpwindow, 11, 15),
                                        ifelse(allele_table_summary_20bp_window$annotation == "5bp_insertion" & 
                                         grepl("plasmid_RNF2", allele_table_summary_20bp_window$sample), 
                                         substring(allele_table_summary_20bp_window$Aligned_Sequence_20bpwindow, 11, 15),
                                        ifelse(allele_table_summary_20bp_window$annotation == "5bp_insertion" & 
                                         grepl("gDNA_HEK3", allele_table_summary_20bp_window$sample), 
                                         substring(allele_table_summary_20bp_window$Aligned_Sequence_20bpwindow, 11, 15),
                                        ifelse(allele_table_summary_20bp_window$annotation == "5bp_insertion" & 
                                         grepl("gDNA_EMX1", allele_table_summary_20bp_window$sample), 
                                         substring(allele_table_summary_20bp_window$Aligned_Sequence_20bpwindow, 11, 15),
                                        ifelse(allele_table_summary_20bp_window$annotation == "5bp_insertion" & 
                                         grepl("gDNA_RNF2", allele_table_summary_20bp_window$sample), 
                                         substring(allele_table_summary_20bp_window$Aligned_Sequence_20bpwindow, 11, 15),
                                         NA))))))

# reverse complement plasmid insertion sequences
allele_table_summary_20bp_window$insertion[!is.na(allele_table_summary_20bp_window$insertion) & 
                               grepl("plasmid", allele_table_summary_20bp_window$sample)] <- unname(sapply(
    allele_table_summary_20bp_window$insertion[!is.na(allele_table_summary_20bp_window$insertion) & 
                                   grepl("plasmid", allele_table_summary_20bp_window$sample)]
    , FUN = function(x) as.character(reverseComplement(DNAString(x)))))
    
write.table(allele_table_summary_20bp_window, "data/allele_table_summary_q30_20bpwindow.txt", row.names = FALSE
            , sep = "\t", quote = FALSE)

In [1]:
allele_table_summary_20bp_window <- read.delim("data/allele_table_summary_q30_20bpwindow.txt")

In [9]:
head(allele_table_summary_20bp_window)

,sample,Aligned_Sequence_20bpwindow,n_deleted,n_inserted,n_mutated,Reads,annotation,insertion
,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<chr>
1,gDNA_RNF2_REP3,TAGTCATTACCTGAGGTGTTCGTTG,0,0,0,957376,unedited,NA
2,gDNA_RNF2_REP1,TAGTCATTACCTGAGGTGTTCGTTG,0,0,0,866101,unedited,NA
3,gDNA_RNF2_REP2,TAGTCATTACCTGAGGTGTTCGTTG,0,0,0,843393,unedited,NA
4,gDNA_HEK3_REP3,ACTGAGCACGTGATGGCAGAGGAAA,0,0,0,122385,unedited,NA
5,gDNA_EMX1_REP2,AGCAGAAGAAGAAGGGCTCCCATCA,0,0,0,93476,unedited,NA
6,gDNA_HEK3_REP2,ACTGAGCACGTGATGGCAGAGGAAA,0,0,0,92782,unedited,NA


In [26]:
dat <- allele_table_summary_20bp_window %>% group_by(sample, annotation) %>% summarise(Reads = sum(Reads))
dat <- dat %>% group_by(sample) %>% mutate(prop = Reads/sum(Reads))
tail(dat)
min(dat$prop[grepl("plasmid", dat$sample) & dat$annotation == '5bp_insertion'])
mean(dat$prop[grepl("plasmid", dat$sample) & dat$annotation == '5bp_insertion'])
max(dat$prop[grepl("plasmid", dat$sample) & dat$annotation == '5bp_insertion'])

`summarise()` has grouped output by 'sample'. You can override using the
`.groups` argument.


sample,annotation,Reads,prop
<chr>,<chr>,<int>,<dbl>
plasmid_RNF2_REP1,unedited,2,1.539077e-05
plasmid_RNF2_REP2,5bp_insertion,121758,7.948947e-01
plasmid_RNF2_REP2,other,31414,2.050857e-01
plasmid_RNF2_REP2,unedited,3,1.958544e-05
plasmid_RNF2_REP3,5bp_insertion,100353,7.936620e-01
plasmid_RNF2_REP3,other,26090,2.063380e-01


[1] 0.793662

[1] 0.8768988

[1] 0.9348134

In [11]:
allele_table_deduplicated <- allele_table_summary_20bp_window[
    allele_table_summary_20bp_window$annotation == '5bp_insertion',]
allele_table_deduplicated <- allele_table_deduplicated[!duplicated(paste(allele_table_deduplicated$sample
                                                                         , allele_table_deduplicated$insertion)),]
allele_table_plot <- spread(allele_table_deduplicated[,c("insertion", 'sample', "Reads")]
                            , key = sample, value = Reads, fill = 1)
allele_table_plot$total <- rowSums(allele_table_plot[,2:length(colnames(allele_table_plot))])
allele_table_plot <- allele_table_plot[order(allele_table_plot$total, decreasing = TRUE),]
write.table(allele_table_plot, "data/allele_matrix_deduplicated_q30_20bpwindow.txt", row.names = FALSE
            , sep = "\t", quote = FALSE)

In [12]:
allele_table_norm <- data.frame(allele_table_plot[,1:19])
rownames(allele_table_norm) <- allele_table_norm$insertion
allele_table_norm <- allele_table_norm[,2:19]

# depth normalize and log transform
allele_table_norm <- sweep(allele_table_norm,2,colSums(allele_table_norm),`/`)
allele_table_norm <- log2(allele_table_norm*1000000)
allele_table_stats <- allele_table_norm

# calculate fold change over mean of plasmid library
allele_table_stats$EMX1_log2FC <- rowMeans(allele_table_norm[
    ,grepl("gDNA_EMX1", colnames(allele_table_norm))]) - rowMeans(
    allele_table_norm[,grepl("plasmid_EMX1", colnames(allele_table_norm))])
allele_table_stats$RNF2_log2FC <- rowMeans(allele_table_norm[
    ,grepl("gDNA_RNF2", colnames(allele_table_norm))]) - rowMeans(
    allele_table_norm[,grepl("plasmid_RNF2", colnames(allele_table_norm))])
allele_table_stats$HEK3_log2FC <- rowMeans(allele_table_norm[
    ,grepl("gDNA_HEK3", colnames(allele_table_norm))]) - rowMeans(
    allele_table_norm[,grepl("plasmid_HEK3", colnames(allele_table_norm))])

# logFC per replicate
allele_table_norm[,grepl("gDNA_EMX1", colnames(allele_table_norm))] <- allele_table_norm[
    ,grepl("gDNA_EMX1", colnames(allele_table_norm))] - rowMeans(
    allele_table_norm[,grepl("plasmid_EMX1", colnames(allele_table_norm))])
allele_table_norm[,grepl("gDNA_HEK3", colnames(allele_table_norm))] <- allele_table_norm[
    ,grepl("gDNA_HEK3", colnames(allele_table_norm))] - rowMeans(
    allele_table_norm[,grepl("plasmid_HEK3", colnames(allele_table_norm))])
allele_table_norm[,grepl("gDNA_RNF2", colnames(allele_table_norm))] <- allele_table_norm[
    ,grepl("gDNA_RNF2", colnames(allele_table_norm))] - rowMeans(
    allele_table_norm[,grepl("plasmid_RNF2", colnames(allele_table_norm))])
allele_table_norm <- allele_table_norm[,grepl("gDNA", colnames(allele_table_norm))]

# zscore
allele_table_norm <- scale(allele_table_norm)

# calculate p value from zscore
allele_table_norm <- data.frame(2*pnorm(-abs(allele_table_norm)))

# combine p values from replicates with Stouffer’s method
allele_table_stats$RNF2_pval <- unname(unlist(sapply(1:nrow(allele_table_norm), function(i) 
    sumz(unlist(allele_table_norm[i,grepl("gDNA_RNF2", colnames(allele_table_norm))]))['p'])))
allele_table_stats$HEK3_pval <- unname(unlist(sapply(1:nrow(allele_table_norm), function(i) 
    sumz(unlist(allele_table_norm[i,grepl("gDNA_HEK3", colnames(allele_table_norm))]))['p'])))
allele_table_stats$EMX1_pval <- unname(unlist(sapply(1:nrow(allele_table_norm), function(i) 
    sumz(unlist(allele_table_norm[i,grepl("gDNA_EMX1", colnames(allele_table_norm))]))['p'])))
write.table(allele_table_stats, "data/allele_matrix_normalized_stats_q30_20bpwindow.txt", row.names = TRUE
            , sep = "\t", quote = FALSE)

In [28]:
allele_table_stats <- read.delim("data/allele_matrix_normalized_stats_q30_20bpwindow.txt")
head(allele_table_stats)

,gDNA_EMX1_REP1,gDNA_EMX1_REP2,gDNA_EMX1_REP3,gDNA_HEK3_REP1,gDNA_HEK3_REP2,gDNA_HEK3_REP3,gDNA_RNF2_REP1,gDNA_RNF2_REP2,gDNA_RNF2_REP3,plasmid_EMX1_REP1,⋯,plasmid_RNF2_REP1,plasmid_RNF2_REP2,plasmid_RNF2_REP3,total,EMX1_log2FC,RNF2_log2FC,HEK3_log2FC,RNF2_pval,HEK3_pval,EMX1_pval
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GTCAG,16.984365,16.951205,16.959148,17.280262,17.270789,17.266018,10.23022,10.35191,10.12793,16.748306,⋯,9.502832,9.94645,10.01902,15.75119,0.21478810,0.4139206,0.40840793,0.59433851,0.4732554,0.7776647005
GCTGC,9.990104,10.033502,9.917822,9.224799,9.348660,10.062302,16.38929,16.40034,16.39287,9.878143,⋯,16.063139,16.08719,16.06068,15.61366,0.06499027,0.3238235,0.64738212,0.69795182,0.2507616,0.8828926205
GGGGG,8.624151,8.610645,8.957715,11.541910,11.566891,11.470561,12.59162,12.68495,12.63889,11.789756,⋯,13.117934,13.10565,13.15147,12.30073,-3.10187130,-0.4865340,-0.46664519,0.84925854,0.8919631,0.0013517462
GCCCC,9.307294,9.502121,9.130551,9.283693,9.289282,9.253444,12.12349,12.09552,12.06461,9.936091,⋯,11.113217,10.93438,10.87317,11.38546,-0.57381611,1.1209530,0.54390318,0.05702057,0.3188213,0.9877331681
GTGGG,7.933836,7.894438,7.424282,11.166532,11.118978,11.144097,11.37848,11.41477,11.41759,11.061148,⋯,12.035564,12.18422,12.24636,11.25089,-3.36086273,-0.7517691,0.05253704,0.54634737,0.9012997,0.0003747302
GCCCG,10.249191,10.263672,10.433743,9.880151,9.786781,10.226874,11.84364,11.67136,11.81920,10.002688,⋯,10.851442,10.76066,10.56651,11.21356,0.22658080,1.0518635,0.38513821,0.07763736,0.5222967,0.7708793752
